<a href="https://colab.research.google.com/github/SaberaTalukder/Chen_Institute_DataSAI_for_Neuroscience/blob/main/07_11_22_day5_single_cell_and_hypothesis_testing/code/diy_notebooks/finalCountColabExercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Modeling and Variance Stabilization of Count Data**

For this problem you will be exploring various models which can be used to describe count data i.e. the gene-count matrices we use in single-cell and investigate the impact of common normalization and variance stabilization practices on the features of those counts.

Single-cell gene counts, which describe stochastically sampled, discrete measurements of counts (or UMIs, Unique Molecular Identifiers), are often modeled as being generated from a negative binomial (or Gamma-Poisson) distribution rather than from the simpler, Poisson distribution.

You will explore how these assumptions and models fit to real datasets.

In [ ]:
#To run a code cell, select the cell and hit Command/Ctrl+Enter or click the run/play symbol
#Click Insert --> Code Cell or the '+ Code' option to insert a new code cell

In [ ]:
#Click Insert --> Text Cell or the '+ Text' option to insert a cell for text as below

In [ ]:
# This is  used to time the running of the notebook
import time
start_time = time.time()

Text here for descriptions, explanations, etc

##**Import data and install packages**

In [ ]:
!pip --quiet install anndata

In [ ]:
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt #Can use other plotting packages like seaborn

import anndata

from scipy import optimize
from scipy.special import gammaln
from scipy.special import psi
from scipy.special import factorial
from scipy.optimize import fmin_l_bfgs_b as optim

In [ ]:
# ! allows you to run commands in the command line, as you would in your normal terminal/command line interface

In [ ]:
# Download MoP BICCN data from Caltech Data 

!wget --content-disposition https://data.caltech.edu/tindfiles/serve/b75ba34a-46ed-458e-8c1f-4e805cf4871a/

In [ ]:
!tar -xf *.gz

Use the function below for fitting negative binomial distributions. 


In [ ]:
# X = numpy array of the data (e.g. 1D array with all the counts for one gene)
# initial params is a numpy array representing the initial values of
# size and prob parameters
# Returns: Dict with 'r' and 'p' fits
def fit_nbinom(X, initial_params=None):
    ''' This code is adapted from https://github.com/gokceneraslan/fit_nbinom
    '''
    infinitesimal = np.finfo(float).eps

    #Calculate and return negative log likelihood for NB
    def log_likelihood(params, *args):
        r, p = params
        X = args[0]
        N = X.size

        # MLE estimate based on the formula on Wikipedia:
        # http://en.wikipedia.org/wiki/Negative_binomial_distribution#Maximum_likelihood_estimation
        result = np.sum(gammaln(X + r)) \
            - np.sum(np.log(factorial(X))) \
            - N * (gammaln(r)) \
            + N * r * np.log(p) \
            + np.sum(X * np.log(1 - (p if p < 1 else 1 - infinitesimal)))

        return -result

    if initial_params is None:
        # reasonable initial values (from fitdistr function in R)
        m = np.mean(X)
        v = np.var(X)
        size = (m ** 2) / (v-m) if v > m else 10

        # convert mu/size parameterization to prob/size
        p0 = size / ((size + m) if size + m != 0 else 1)
        r0 = size
        initial_params = np.array([r0, p0])

    bounds = [(infinitesimal, None), (infinitesimal, 1)]
    optimres = optim(log_likelihood,
                     x0=initial_params,
                     args=(X,),
                     approx_grad=1,
                     bounds=bounds)

    params = optimres[0]
    return {'r': params[0], 'p': params[1]}

## **Reading in the Data**

**The dataset**

We will be working with 10x Genomics sequencing data from the mouse primary cortex, produced by the BRAIN Initiative Cell Census Network (BICCN) [(Yao et al. 2021)](https://www.nature.com/articles/s41586-021-03500-8). They assayed 94,162 cells by 10x sequencing (as well as SMART-seq), though we will work with only a subsetted sample today.  The cell type labels include GABAergic and Glutamatergic cells as well as Non-Neuronal cells, and were sampled from both sexes.

<center><img src="https://drive.google.com/uc?export=view&id=1TKA0m3BEox8aBpoSNFMnIb3yftCfWyM4" alt="EMFigure" width="350" height="350"><center>


**The Count matrix**

Count matrix has 7,897 cells with counts for 2000 genes. Though greater numbers of genes were measured, the dataset is subsetted to 2000 genes for ease of analysis. For this exercise we are not using data from both male and female mice (only female mice).

The 2000 genes were selected by filtering for genes expressed in greater than 1% of cells, and subsetting to 2000 of those genes.


 


In [ ]:
#Get gene count matrix
import scipy.io as sio

X = sio.mmread('mop_count_mat.mtx')

X.shape

## **Assessing Count Distribution Fits to Data** 

The Poisson distribution expresses the probability of some $x$ number of events occurring in a fixed interval of time/space, with these events occuring at a known constant mean rate $\lambda$ (and independently of time since the last event).

The Poisson model is
\begin{align}
f(x; \lambda)= \Pr(X{=}x)= \frac{\lambda^x e^{-\lambda}}{x!}.
\end{align}

The negative binomial (NB) distribution describes the probability of seeing $x$ failures until some $r$ successes have occurred, with $p$ denoting the probability of success. 'Successes' can be thought of read counts here i.e the event that a read is a count for a given gene.

The NB model ($\text{NB}
(r,p)$) is
\begin{align}
f(x;r,p)\equiv \Pr(X=x)={\binom {x+r-1}{r-1}}(1-p)^{x}p^{r}
\end{align}

The Poisson distribution can be written as special case of the NB where $ \operatorname {Poisson} (\lambda )=\lim _{r\to \infty }\operatorname {NB} \left(r,{\frac {\lambda }{r+\lambda }}\right)$, with $p ={\frac {\lambda }{r+\lambda }}$.


Note: We can also denote $r,p$ as $\mu,\phi$ where $\mu$ represent the mean and $\phi$ represents the dispersion parameter. Here $p = \dfrac{r}{r + \mu}$ and $r = \dfrac{1}{\phi}$.
This convention is also common in the single-cell literature.

### **a) Plot mean versus variance of expression for all genes (across all cells) and comment on what trends you notice between the two (e.g. overdispersion etc).**
You may want to use a log-log scale for plotting (e.g. plt.loglog())

### **b) Fit Poisson and NB models for the first 10 genes.**

Fit and save the parameters for these two models for the first 10 genes $g$.

For the Poisson models we can use the MLE estimate for $\lambda$ which is $\dfrac{1}{n}\sum_{i=1}^n x_{i,g}$.

For the NB $r \text{ and } p$ cannot (concurrently) be derived analytically. You can use the **fit_nbinom(X)** function defined above to obtain $r \text{ and } p$ fits for each gene.

**Report the parameter fits for the first 10 genes.**

### **c) Perform a likelihood ratio test, and calculate the resulting p-value, between these two models (for a single gene at index=5), and comment on the implications of these outputs.**

We will be testing whether the added parameters of the NB improve the fit of the model to the data versus the Poisson model (which is a nested version of the NB), or not (if both models are equal). Thus the test is  
\begin{align}
H_0 : r = ∞ \\
H_1: r < ∞.
\end{align}

For some gene $g$ we can calculate the log-likelihood for both models given parameter fits, and take the ratio of those likelihoods (which is the difference between the log-likelihoods).




$-2log(\dfrac{L_{H_0}}{L_{H_1}}) = -2log(\dfrac{L_{Poisson}}{L_{NB}})$ asymptotically approaches the chi-squared $χ^2$ distribution under the null hypothesis (in this case for a degree of freedom (df) of 1, the difference in the dfs of the models). Thus the cdf of the $χ^2$ distribution can be used to calculate a p-value given $-2 *$ likelihood ratio (using [stats.chi2.cdf](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2.html)). 

For the NB model the likelihood (for gene $g$) is:
\begin{align}
L^{(g)}(r,p)=\prod _{i=1}^{N}f(x_{i,g};r,p)
\end{align}

And the log-likelihood:
\begin{align}
\ell^{(g)} (r,p)=\sum _{i=1}^{N}\ln(\Gamma (x_{i,g}+r))-\sum _{i=1}^{N}\ln(x_{i,g}!)-N\ln(\Gamma (r))+\sum _{i=1}^{N}x_{i,g}\ln(1-p)+Nr\ln(p).
\end{align} where $ln(\Gamma)$ can be calculated with [gammaln](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gammaln.html).


For the Poisson model the likelihood (for gene $g$) is:
\begin{align}
L^{(g)}(\lambda ) =\prod _{i=1}^{n}f(x_{i,g}\mid \lambda ).
\end{align}

And the log-likelihood:
\begin{align}
\ell^{(g)} (\lambda )= -n\lambda +\left(\sum _{i=1}^{n}x_{i,g}\right)\ln(\lambda )-\sum _{i=1}^{n}\ln(x_{i,g}!).
\end{align}


**Explicitly calculate and report the log-likelihood ratio for the gene at the 5th position only (the 6th gene since python is zero-indexed). Additionally report the p-value for this ratio (using the chi-squared cdf) and comment on what this value means in terms of the null and the alternative hypothesis.**

You can also use the code for the negative binomial likelihood formula already given in the fit_nbinom function.

In [ ]:
from scipy.stats import chi2


## **Variance Stabilization and Normalization of Counts**

For our purposes, we will use the $\mu,\phi$ parametrization of the negative binomial (NB) for this problem. Here $\phi$ is the dispersion and $\mu$ is the mean. 

In this configuration, $\operatorname {var}(X) = \mu + \phi\mu^2$ (unlike the Poisson where $\operatorname{var}(X) = \mu$). $x_i$ represents expression of gene $i$.


As described in the assignment, we can find a variance-stabilizing transform, where given
\begin{align}
\operatorname {var} (X)=h(\mu ),\,
\end{align}
a suitable transform would be
\begin{align}
 y\propto \int ^{x}{\frac {1}{\sqrt {h(\mu )}}}\,d\mu 
\end{align}
to result in a constant (mean-independent) variance.

This is useful when performing various analyses, such as linear regression, which assume homoscedascity (i.e. all genes/features have the same variance).


### **e) Run the log1p, Pearson residual, and $\mathbf{\text{sinh}^{-1}}$ variance stabilization transforms on the full dataset. We will use a $\phi=0.01$**

Below you will test out the effect of common variance-stabilization procedures.

[In 1948](https://academic.oup.com/biomet/article-abstract/35/3-4/246/280278?redirectedFrom=fulltext), Frank Anscombe developed several transformations for the Poisson and NB distributions including

\begin{align}
y \propto \dfrac{\text{sinh}^{-1}(\sqrt{\phi x_i})}{\sqrt{\phi}} \mathbf{\tag{1}}
\end{align}and
\begin{align}
y \propto \text{log}(x_i+\dfrac{1}{2\phi}) \tag{2}
\end{align} (similar to the log1p we've seen before) which can approximate the $\text{sinh}^{-1}$ solution.

The log1p approximation is then
\begin{align}
y \propto \text{log}(x_i+1) \mathbf{\tag{3}}
\end{align} 



Another common method is to use Pearson residuals, shown below:

\begin{align}
y \propto \dfrac{x_i − \mu_i}{\sqrt{\mu_i + \phi \mu_i^2}}. \mathbf{\tag{4}}
\end{align}

Again $x_i$ represents expression of gene $i$.

**After running each transformation, print *only* the transformed values for the first gene, for the first 10 cells, under each transform (1,3 and 4).**

### **f) For each of the three transformation methods, make a single plot of the variance ($\sigma^2$) versus the mean ($\mu$) for all genes, and comment on the trends you notice (particularly compared to a).**